In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import UTC
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv("../../db/data/merge/secondary/logs_.csv", index_col=0)
df.index = pd.to_datetime(df.index)
datetime(2022, 7, 1, 0, 0, 0, 0, UTC)

df = df.loc[datetime(2010, 6, 1, 0, 0, 0, 0, UTC):datetime(2022, 7, 1, 0, 0, 0, 0, UTC)]
display(df.head(1), df.tail(1))
df.shape


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2010-06-01 00:00:00+00:00,-0.195535,-0.431203,-0.701336,-0.140981,-0.470019,-0.252915,-0.504554,-0.271715,0.354467,0.168679,...,-0.372762,0.185407,-0.157289,-0.047979,-0.305857,-0.55999,-0.335956,0.293033,0.037194,-0.217444


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,-0.05858,-0.048583,0.019215,0.010852,-0.092832,0.009434,0.079595,0.068199,0.036857,-0.023727,...,0.065983,0.070777,-0.053466,-0.080965,-0.077221,-0.004721,-0.121889,0.041164,0.052337,0.119222


(74593, 28)

In [3]:
SYMBOLS = [ 'AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 
            'CAD_CHF', 'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 
            'EUR_CHF', 'EUR_GBP', 'EUR_JPY', 'EUR_NZD', 'EUR_USD', 
            'GBP_AUD', 'GBP_CAD', 'GBP_CHF', 'GBP_JPY', 'GBP_NZD', 
            'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY', 'NZD_USD', 
            'USD_CAD', 'USD_CHF', 'USD_JPY'
          ]



In [4]:
def mean_prediction(df, rate=24):
    data = pd.DataFrame([])
    for sym in SYMBOLS:
        data[f'{sym}_tend'] = df[sym].rolling(rate).mean()
    data.index = df.index
    return data
        


In [5]:
def random_prediction(df):
    data = pd.DataFrame([])
    
    for sym in SYMBOLS:
        data[f'{sym}_pred_vol']=pd.DataFrame(np.random.uniform(low=-0.8,high=0.8,size=(df.shape[0],1)))
    data.index = df.index
    return data

In [6]:
one = random_prediction(df)
two = mean_prediction(df)


In [86]:
def condition(x):
    if x>0:
        return 1
    elif x<0:
        return -1
    else:
        return 0

for sym in two.columns:
    two[sym] = two[sym].map( lambda x: condition(x))
    
# two.reset_index(inplace=True, drop=True)

display(one.shape, two.shape)


(74593, 28)

(74593, 28)

In [8]:
df = two.join(one)

In [9]:
df

,AUD_CAD_tend,AUD_CHF_tend,AUD_JPY_tend,AUD_NZD_tend,AUD_USD_tend,CAD_CHF_tend,CAD_JPY_tend,CHF_JPY_tend,EUR_AUD_tend,EUR_CAD_tend,...,GBP_JPY_pred_vol,GBP_NZD_pred_vol,GBP_USD_pred_vol,NZD_CAD_pred_vol,NZD_CHF_pred_vol,NZD_JPY_pred_vol,NZD_USD_pred_vol,USD_CAD_pred_vol,USD_CHF_pred_vol,USD_JPY_pred_vol
2010-06-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.062804,0.600790,-0.481272,0.646539,0.654128,-0.360478,0.068205,-0.562404,0.692977,0.511683
2010-06-01 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.213869,0.459930,0.552903,0.469836,-0.008888,0.205371,-0.254618,-0.656621,-0.264776,0.069869
2010-06-01 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.263037,-0.547669,-0.388982,-0.293271,-0.236982,0.218049,-0.497861,0.389630,0.616105,0.081335
2010-06-01 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.757506,0.769055,0.494207,0.554903,0.460122,-0.165932,0.338891,0.089363,-0.574242,-0.156346
2010-06-01 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.667671,0.265322,0.434468,0.283295,0.413238,-0.314015,-0.493075,-0.331642,-0.127408,0.318837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30 20:00:00+00:00,0.006474,0.014114,-0.009765,0.001847,0.016264,0.005452,-0.019866,-0.024633,0.003651,0.010634,...,-0.720927,0.748314,-0.594110,-0.104245,0.267943,0.282826,-0.790053,-0.771931,0.385350,0.063807
2022-06-30 21:00:00+00:00,0.008587,0.014815,-0.007723,-0.001280,0.016023,0.007028,-0.015647,-0.025189,-0.000274,0.008625,...,-0.642384,0.351512,0.018658,0.415886,-0.293765,0.369746,0.143886,-0.789369,0.578774,-0.781403
2022-06-30 22:00:00+00:00,0.005350,0.011018,-0.012120,0.002752,0.011124,0.006297,-0.017178,-0.025889,0.002141,0.007053,...,0.075883,0.099213,-0.269009,-0.261851,0.501966,0.171686,-0.603653,0.423809,-0.647470,0.547959
2022-06-30 23:00:00+00:00,0.008782,0.015841,-0.009898,-0.000225,0.013547,0.007478,-0.018268,-0.028147,-0.000604,0.007764,...,0.569492,0.115017,0.154355,0.460274,-0.299520,-0.446707,-0.328095,-0.413485,-0.030076,-0.142608


In [59]:
pred_last = one.iloc[-120:]
pred = one.iloc[-1:]
display(pred_last.tail(1), pred)


,AUD_CAD_pred_vol,AUD_CHF_pred_vol,AUD_JPY_pred_vol,AUD_NZD_pred_vol,AUD_USD_pred_vol,CAD_CHF_pred_vol,CAD_JPY_pred_vol,CHF_JPY_pred_vol,EUR_AUD_pred_vol,EUR_CAD_pred_vol,...,GBP_JPY_pred_vol,GBP_NZD_pred_vol,GBP_USD_pred_vol,NZD_CAD_pred_vol,NZD_CHF_pred_vol,NZD_JPY_pred_vol,NZD_USD_pred_vol,USD_CAD_pred_vol,USD_CHF_pred_vol,USD_JPY_pred_vol
2022-07-01 00:00:00+00:00,0.096069,0.584921,0.573113,0.257581,0.365127,0.508471,-0.38409,-0.713059,0.109561,0.50396,...,-0.495222,0.018256,0.156184,0.025331,0.719274,-0.488226,-0.427949,-0.368498,0.178666,0.645403


,AUD_CAD_pred_vol,AUD_CHF_pred_vol,AUD_JPY_pred_vol,AUD_NZD_pred_vol,AUD_USD_pred_vol,CAD_CHF_pred_vol,CAD_JPY_pred_vol,CHF_JPY_pred_vol,EUR_AUD_pred_vol,EUR_CAD_pred_vol,...,GBP_JPY_pred_vol,GBP_NZD_pred_vol,GBP_USD_pred_vol,NZD_CAD_pred_vol,NZD_CHF_pred_vol,NZD_JPY_pred_vol,NZD_USD_pred_vol,USD_CAD_pred_vol,USD_CHF_pred_vol,USD_JPY_pred_vol
2022-07-01 00:00:00+00:00,0.096069,0.584921,0.573113,0.257581,0.365127,0.508471,-0.38409,-0.713059,0.109561,0.50396,...,-0.495222,0.018256,0.156184,0.025331,0.719274,-0.488226,-0.427949,-0.368498,0.178666,0.645403


In [57]:
def pairs_correlation(df, rate=120):
    data = pd.DataFrame([])
    for pair in df.columns:
        for second_pair in df.columns:
            data[f'{pair}_{second_pair}_corr'] = df[f'{pair}'].rolling(120).corr(df[f'{second_pair}'])
    data.index = df.index
    return data

cor = pairs_correlation(pred_last)
cor = cor.iloc[-1:]





# data = pd.DataFrame([])
# for sym in SYMBOLS:

    
    
    

/tmp/ipykernel_3354/1364896319.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[f'{pair}'].rolling(120).corr(df[f'{second_pair}'])
/tmp/ipykernel_3354/1364896319.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[f'{pair}'].rolling(120).corr(df[f'{second_pair}'])
/tmp/ipykernel_3354/1364896319.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Co

In [93]:
cor = cor.T
pred



,AUD_CAD_pred_vol,AUD_CHF_pred_vol,AUD_JPY_pred_vol,AUD_NZD_pred_vol,AUD_USD_pred_vol,CAD_CHF_pred_vol,CAD_JPY_pred_vol,CHF_JPY_pred_vol,EUR_AUD_pred_vol,EUR_CAD_pred_vol,...,GBP_JPY_pred_vol,GBP_NZD_pred_vol,GBP_USD_pred_vol,NZD_CAD_pred_vol,NZD_CHF_pred_vol,NZD_JPY_pred_vol,NZD_USD_pred_vol,USD_CAD_pred_vol,USD_CHF_pred_vol,USD_JPY_pred_vol
2022-07-01 00:00:00+00:00,0.096069,0.584921,0.573113,0.257581,0.365127,0.508471,-0.38409,-0.713059,0.109561,0.50396,...,-0.495222,0.018256,0.156184,0.025331,0.719274,-0.488226,-0.427949,-0.368498,0.178666,0.645403


In [80]:
def weighted_volatility(volatility):
    weight = pd.DataFrame([])
    
    weight = volatility.apply(lambda x: x / volatility.mean() for x in volatility)
    return  weight

In [101]:


pred = pred.T
print(pred["2022-07-01 00:00:00+00:00"].mean())
# display(pred, w )

0.07424471614219515


In [102]:
w = weighted_volatility(pred)

In [112]:
w

,2022-07-01 00:00:00+00:00
,<lambda>
,2022-07-01 00:00:00+00:00
AUD_CAD_pred_vol,1.293945
AUD_CHF_pred_vol,7.878282
AUD_JPY_pred_vol,7.719245
AUD_NZD_pred_vol,3.469351
AUD_USD_pred_vol,4.917885
CAD_CHF_pred_vol,6.848580
CAD_JPY_pred_vol,-5.173292
CHF_JPY_pred_vol,-9.604168
